![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx.ai python SDK to manage Prompt Template assets and create deployment

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate support for Prompt Template inference and their deployments.

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The goal of this notebook is to demonstrate how to create a Prompt Template asset and deployment pointing on it. In general, a Prompt Template is a pattern for generating prompts for language models. A template may contain instruction, input/output prefixes, few-shot examples and appropriate context that may vary depending on different tasks.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Prompt Template Inference](#prompt)
- [Prompt Template Deployment](#deployment)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials


### Install dependecies

In [ ]:
!pip install "ibm-watsonx-ai>=0.2.3" | tail -n 1

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [2]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [3]:
credentials = {
    "username": username,
    "apikey": api_key,
    "url": url,
    "instance_id": 'openshift',
    "version": '4.8'
}

Alternatively you can use `username` and `password` to authenticate WML services.

```python
credentials = {
    "username": ***,
    "password": ***,
    "url": ***,
    "instance_id": 'openshift',
    "version": '4.8'
}

```

### Defining the project id
The Prompt Template requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [4]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="prompt"></a>
## Prompt Template on `watsonx.ai`

In [5]:
from ibm_watsonx_ai.foundation_models.prompts import PromptTemplateManager, PromptTemplate
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, DecodingMethods, PromptTemplateFormats
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

### Instantiate PromptTemplateManager

In [6]:
prompt_mgr = PromptTemplateManager(
    credentials=credentials,
    project_id=project_id
    )

### Create a Prompt Template object and store it in the project


We use a `PromptTemplate` object to store the properties of a newly created prompt template. Prompt text is composed of the instruction, input/output prefixes, few-shot examples and input text. All of the mentioned fields may contain placeholders (`{...}`) with input variables and for the template to be valid, these input variables must be also specified in `input_variables` parameter.

In [7]:
prompt_template = PromptTemplate(name="New prompt",
                                 model_id=ModelTypes.FLAN_T5_XXL,
                                 model_params = {GenParams.DECODING_METHOD: "sample"},
                                 description="My example",
                                 task_ids=["generation"],
                                 input_variables=["object"],
                                 instruction="Answer on the following question",
                                 input_prefix="Human",
                                 output_prefix="Assistant",
                                 input_text="What is {object} and how does it work?",
                                 examples=[["What is a loan and how does it work?", 
                                            "A loan is a debt that is repaid with interest over time."]]
                                )

Using `store_prompt(prompt_template_id)` method, one can store newly created prompt template in your ptoject.

In [8]:
stored_prompt_template = prompt_mgr.store_prompt(prompt_template=prompt_template)

In [9]:
print(f"Asset id: {stored_prompt_template.prompt_id}")
print(f"Is it a template?: {stored_prompt_template.is_template}")

Asset id: 87e3587a-8f38-4297-8c5b-575c9b6b3f40
Is it a template?: True


We can also store a template which is a `langchain` Prompt Template object.

In [ ]:
!pip install langchain | tail -n 1

In [15]:
from langchain.prompts import PromptTemplate as LcPromptTemplate

langchain_prompt_template = LcPromptTemplate(template="What is {object} and how does it work?",
                                             input_variables=["object"])
stored_prompt_template_lc = prompt_mgr.store_prompt(prompt_template=langchain_prompt_template)
print(f"Asset id: {stored_prompt_template_lc.prompt_id}")

Asset id: fc6f6d1b-e30f-4909-9a41-519b59ae6915


### Manage Prompt Templates

In [16]:
prompt_mgr.list()

,ID,NAME,CREATED,LAST MODIFIED
0,fc6f6d1b-e30f-4909-9a41-519b59ae6915,My prompt,2024-03-27T14:16:03Z,2024-03-27T14:16:03Z
1,78c7f06a-f72c-42af-9dda-031a0dd94745,My prompt,2024-03-27T14:15:36Z,2024-03-27T14:15:36Z
2,87e3587a-8f38-4297-8c5b-575c9b6b3f40,New prompt,2024-03-27T14:14:47Z,2024-03-27T14:14:47Z


To retrive Prompt Template asset from project and return string that contains Prompt Template input we use `load_prompt(prompt_template_id, astype=...)`. Returned input string is composed of the fields: `instruction`, `input_prefix`, `output_prefix`, `examples` and `input_text`. After substituting prompt variables, one can evaluate a language model on the obtained string.

In [17]:
prompt_text = prompt_mgr.load_prompt(prompt_id=stored_prompt_template.prompt_id, astype=PromptTemplateFormats.STRING)
print(prompt_text)

Answer on the following question

Human What is a loan and how does it work?
Assistant A loan is a debt that is repaid with interest over time.

Human What is {object} and how does it work?
Assistant


To update Prompt Template data use `prompt_mgr.update_prompt` method.

In [18]:
prompt_with_new_instruction = PromptTemplate(instruction="Answer on the following question in a concise way.")
prompt_mgr.update_prompt(prompt_id=stored_prompt_template.prompt_id, 
                         prompt_template=prompt_with_new_instruction)
prompt_text = prompt_mgr.load_prompt(prompt_id=stored_prompt_template.prompt_id, astype=PromptTemplateFormats.STRING)
print(prompt_text)

Answer on the following question in a concise way.

Human What is a loan and how does it work?
Assistant A loan is a debt that is repaid with interest over time.

Human What is {object} and how does it work?
Assistant


Furthermore, to get information about locked state of Prompt Template run following method

In [19]:
prompt_mgr.get_lock(prompt_id=stored_prompt_template.prompt_id)

{'locked': True, 'locked_by': '1000330999', 'lock_type': 'edit'}

Using `lock` or `unlock` method, one can change locked state of Prompt Template asset.

In [20]:
prompt_mgr.unlock(prompt_id=stored_prompt_template_lc.prompt_id)

{'locked': False}

Once the prompt template is unlocked it can be deleted. You can also use the `list` method to check the available prompt templates (passing `limit=2` parameter will return only 2 recently created templates).

In [21]:
print(f"Id of the Prompt Template asset that will be deleted: {stored_prompt_template_lc.prompt_id}")
prompt_mgr.delete_prompt(prompt_id=stored_prompt_template_lc.prompt_id)
prompt_mgr.list(limit=2)

Id of the Prompt Template asset that will be deleted: fc6f6d1b-e30f-4909-9a41-519b59ae6915


,ID,NAME,CREATED,LAST MODIFIED
0,78c7f06a-f72c-42af-9dda-031a0dd94745,My prompt,2024-03-27T14:15:36Z,2024-03-27T14:15:36Z
1,87e3587a-8f38-4297-8c5b-575c9b6b3f40,New prompt,2024-03-27T14:14:47Z,2024-03-27T14:16:15Z


<a id="deployment"></a>
## Deployment pointing to Prompt Template

To create deployment pointing to a Prompt template asset we have to initialized `APIClient` object.

In [22]:
from ibm_watsonx_ai import APIClient

client = APIClient(wml_credentials=credentials)
client.set.default_project(project_id)

'SUCCESS'

In the deployment exmaple we will use the prompt with the following input

In [23]:
prompt_input_text = prompt_mgr.load_prompt(prompt_id=stored_prompt_template.prompt_id, 
                                           astype=PromptTemplateFormats.STRING)
print(prompt_input_text)

Answer on the following question in a concise way.

Human What is a loan and how does it work?
Assistant A loan is a debt that is repaid with interest over time.

Human What is {object} and how does it work?
Assistant


Now, we create deployment providing the id of Prompt Template asset and meta props. 

In [24]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "SAMPLE DEPLOYMENT PROMPT TEMPLATE",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.BASE_MODEL_ID: "ibm/granite-13b-chat-v2"}

deployment_details = client.deployments.create(artifact_uid=stored_prompt_template.prompt_id, meta_props=meta_props)



#######################################################################################

Synchronous deployment creation for uid: '87e3587a-8f38-4297-8c5b-575c9b6b3f40' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='59628611-e1fc-4575-bc11-80617a2c3666'
------------------------------------------------------------------------------------------------




In [25]:
client.deployments.list()

------------------------------------  ---------------------------------  -----  ------------------------  ----------------  ----------  ----------------
GUID                                  NAME                               STATE  CREATED                   ARTIFACT_TYPE     SPEC_STATE  SPEC_REPLACEMENT
59628611-e1fc-4575-bc11-80617a2c3666  SAMPLE DEPLOYMENT PROMPT TEMPLATE  ready  2024-03-27T14:16:34.173Z  foundation_model
------------------------------------  ---------------------------------  -----  ------------------------  ----------------  ----------  ----------------


,GUID,NAME,STATE,CREATED,ARTIFACT_TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,59628611-e1fc-4575-bc11-80617a2c3666,SAMPLE DEPLOYMENT PROMPT TEMPLATE,ready,2024-03-27T14:16:34.173Z,foundation_model,,


Substitute prompt variables and generate text

In [26]:
deployment_id = deployment_details.get("metadata", {}).get("id")

In [27]:
client.deployments.generate_text(deployment_id, params={"prompt_variables": {"object": "a mortgage"},
                                                        GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
                                                        GenParams.STOP_SEQUENCES: ['\n\n'],
                                                        GenParams.MAX_NEW_TOKENS: 50})

Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.


' A mortgage is a type of loan used to buy a home. When you take out a mortgage, you borrow money from a bank or other financial institution to pay for the home. The home serves as collateral for the loan, which means that if you'

### Generate text using ModelInference

You can also generate text based on your Prompt Template deployment using `ModelInference` class.

In [28]:
from ibm_watsonx_ai.foundation_models import ModelInference

In [29]:
model_inference = ModelInference(deployment_id=deployment_id, 
                                 credentials=credentials, 
                                 project_id=project_id)

Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.


In [30]:
model_inference.generate_text(params={"prompt_variables": {"object": "a mortgage"},
                                      GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
                                      GenParams.STOP_SEQUENCES: ['\n\n'],
                                      GenParams.MAX_NEW_TOKENS: 50})

' A mortgage is a type of loan used to buy a home. When you take out a mortgage, you borrow money from a bank or other financial institution to pay for the home. The home serves as collateral for the loan, which means that if you'

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to create valid Prompt Template and store it in watsonx.ai project. Furthermore, you also learned how to create deployment pointing to a Prompt Template asset and generate text using base model.
 
 Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Mateusz Świtała**, Software Engineer at Watson Machine Learning.

**Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2023, 2024 IBM. This notebook and its source code are released under the terms of the MIT License.